In [ ]:
#Import des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import geopandas as gpd
%matplotlib inline

from scipy.stats import pearsonr
import statsmodels.api

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

sns.set_theme(style='whitegrid', palette='pastel')
#sns.set_theme(style = "ticks", context = "talk", palette = "bright")
#sns.color_palette("Paired")

In [ ]:
#Import des DataFrames
commune=pd.read_parquet('donnee-comm-data.gouv-parquet-2023-geographie2024-produit-le2024-07-05.parquet')
dep=pd.read_csv('donnee-dep-data.gouv-2023-geographie2024-produit-le2024-07-05.csv',sep=';')
reg=pd.read_csv('donnee-reg-data.gouv-2023-geographie2024-produit-le2024-07-05.csv',sep=';')
commune.tail(10)

# I) Nettoyage des données dans les différents DataFrames
## A) Vérification de la cohérence et de l'uniformité des données
### 1) Vérification de la cohérence
Tout d'abord, il convient de vérifier que chaque variable est au bon type. L'objectif est d'avoir une uniformité des types de valeurs entre les 3 DataFrames (commune, dep et reg). 
A noter que les dates seront traitées séparément car elles font l'objet d'une traitement particulier


In [ ]:
## DataFrame commune
#les colonnes classe et unité.de.compte doivent être au format object
commune['classe']=commune['classe'].astype('str')
commune['unité.de.compte']=commune['unité.de.compte'].astype('str')
commune.info()

In [ ]:
## DataFrame dep
#La colonne taux pour mille n'est pas au bon type, il convient de changer le type object au type float
#Pour cela, il est nécessaire de changer chaque virgule par un point. 
#On en profitera pour arrondir les valeurs à 10-3

dep['tauxpourmille']=dep['tauxpourmille'].str.replace(',', '.')
dep['tauxpourmille']=dep['tauxpourmille'].astype(float)
dep['tauxpourmille']=dep['tauxpourmille'].round(3)
dep.info()


In [ ]:
## DataFrame reg
#La colonne taux pour mille n'est pas au bon type, il convient de changer le type object au type float
reg['tauxpourmille']=reg['tauxpourmille'].str.replace(',', '.')
reg['tauxpourmille']=reg['tauxpourmille'].astype(float)
reg['tauxpourmille']=reg['tauxpourmille'].round(3)
reg.info()

### 2) Vérification de l'uniformité des données
Il convient de vérifier que les données sont uniformes, c'est à dire qu'elles doivent être écrites toujours de la même manière en respectant une règle unique
#### a) Données quantitatives

In [ ]:
reg.head(1)

In [ ]:
## DataFrame commune
commune['annee'].unique() #Toutes les années sont écrites au même format
commune['classe'].unique() #Toutes les classes sont écrites au même format et il n'y en a pas deux identiques écrites différemment 
commune['unité.de.compte'].unique() #Ici victime et victime entendue correspondent a la même unité de compte. Il faut donc modifier cela 
commune['unité.de.compte']=commune['unité.de.compte'].replace('victime entendue','victime')
commune['valeur.publiée'].unique() #Toutes les valeurs publiées sont écrites au même format 

## DataFrame dep
dep['annee'].unique() #Toutes les années sont écrites au même format
dep['classe'].unique() #Toutes les années sont écrites au même format
dep['Code.département'].unique() #Tous les départements sont écrits au même format doublon. A noter qu'il est normal que la valeur "20" soit manquante puisqu'il s'agit de la corse, dont le code a été remplacé en 1976 par 2A et 2B
dep['unité.de.compte'].unique() #Ici victime et victime entendue correspondent a la même unité de compte. Il faut donc modifier cela 
dep['unité.de.compte']=dep['unité.de.compte'].replace('victime entendue','victime')

## DataFrame reg
reg['annee'].unique() #Toutes les années sont écrites au même format
reg['classe'].unique() #Toutes les années sont écrites au même format
reg['Code.région'].unique() #Toutes les régions sont présentes et sont écrites au même format 

#### b) Dates
L'étude des formats a démontré que les dates (ici "annee") sont au format int, ce qui est incorrect. Pour la suite de l'analyse, ces variables doivent être au format datetime64. 

In [ ]:
commune['annee'] = pd.to_datetime(commune['annee'].apply(lambda x: f"20{x:02d}")).dt.year
dep['annee'] = pd.to_datetime(dep['annee'].apply(lambda x: f"20{x:02d}")).dt.year
reg['annee'] = pd.to_datetime(reg['annee'].apply(lambda x: f"20{x:02d}")).dt.year

In [ ]:
commune.info()

## B) Vérifications des doublons

In [ ]:
# Recherche de doublons 
print('Il y a',commune.duplicated().sum(),'doublons dans le df commune')
print('Il y a',dep.duplicated().sum(),'doublons dans le df dep')
print('Il y a',reg.duplicated().sum(),'doublons dans le df reg')

## C) Vérification des valeurs manquantes
### 1) Analyse globale

In [ ]:
# Recherche de valeurs manquantes du df departement 
print('Il y a',dep.isna().any(axis = 0).sum(),'colonnes contenant des valeurs manquantes dans le df departement')
# Recherche de valeurs manquantes sur les lignes du df département
#dep.isna().any(axis = 1).sum() Inutile donc 

In [ ]:
# Recherche de valeurs manquantes du df région
print('Il y a',reg.isna().any(axis = 0).sum(),'colonnes contenant des valeurs manquantes dans le df reg')
# Recherche de valeurs manquantes sur les lignes du df régions
#reg.isna().any(axis = 1).sum() Inutile donc 

In [ ]:
# Recherche de valeurs manquantes sur les colonnes du df commune
print('Il y a',commune.isna().any(axis = 0).sum(),'colonnes contenant des valeurs manquantes dans le df commune') 
# Recherche de valeurs manquantes sur les lignes du df communes
print('Il y a',commune.isna().any(axis = 1).sum(),'lignes contenant des valeurs manquantes sur',commune.shape[0],'lignes sur le df.')
print('Les colonnes concernées sont les colonnes faits, taux pour mille, complementinfoval, complementinfototaux')

### 2) Traitement des valeurs manquantes
Le fichier des métadonnées du dataset indique que si la variable contenue dans la colonne "valeur.publiée" vaut ndiff, alors les variables contenues dans la colonne "faits" et "tauxpourmille" ne seront pas renseignées. De ce fait, il a été décidé de supprimer toutes les lignes donc la colonne "valeur.publiée" vaut ndiff. 

In [ ]:
#Suppression des valeurs manquantes dans la colonne "faits"
commune.loc[commune['valeur.publiée'] == 'ndiff', 'faits'] = 0.0

#Suppression des valeurs manquantes dans la colonne "tauxpourmille"
commune.loc[commune['valeur.publiée'] == 'ndiff', 'tauxpourmille'] = 0.0
commune.isna().any(axis = 0)
#On  observe qu'il reste des NaN. On les localise et on étudie
commune.loc[commune[['tauxpourmille']].isna().any(axis=1)]
#Les NAN restants peuvent être recalculés car le taux pour mille est le nombre de faits divisé par le nombre d'habitants multiplié par 1000 
commune['millPOP'].unique() #On vérifie que millPOP ne prend pas la valeur 0 auquel cas la division serait impossible 
commune['tauxpourmille']=commune['tauxpourmille'].fillna((commune['faits']/commune['POP'])*1000)

#Il reste des NaNs dans taux pour mille, il s'agit des villes dans lesquelles il y a 0 habitants. 
commune['tauxpourmille']=commune['tauxpourmille'].fillna(0) 
#commune = commune.dropna(axis = 0, how = 'all', subset = ['tauxpourmille']) 

display(pd.DataFrame(commune.isna().sum(), columns=["Nombre de NA"]))  

#Les colonnes "complementinfoval" et "complementinfototaux" contiennent encore des NaN mais ne sont pas utiles pour l'étude. 

Certaines colonnes ne sont pas utiles pour cette étude. Il convient donc de les supprimer. 
Si dans le futur il s'avère qu'il est nécessaire de les garder, il suffira de supprimer la cellule suivante.

In [ ]:
## DataFrame commune
commune=commune.drop("complementinfoval", axis=1)
commune=commune.drop("complementinfotaux", axis=1)
commune=commune.drop("LOG", axis=1)
commune=commune.drop("millLOG", axis=1)
commune=commune.drop("millPOP", axis=1)
commune=commune.drop("valeur.publiée", axis=1)
## DataFrame reg
reg=reg.drop("LOG", axis=1)
reg=reg.drop("millLOG", axis=1)
reg=reg.drop("millPOP", axis=1)
## DataFrame dep
dep=dep.drop("LOG", axis=1)
dep=dep.drop("millLOG", axis=1)
dep=dep.drop("millPOP", axis=1)




## D) Autres préprocessings
Renommage des colonnes pour une meilleure lisibilité. 

In [ ]:
## DataFrame commune
dictionnaire = {'CODGEO_2024': 'codgeo',
                'unité.de.compte': 'cible',
                'valeur.publiée': 'publication',
                'POP':'pop',
                'millPOP':'millpop'}
commune=commune.rename(dictionnaire, axis = 1) 
## DataFrame reg
dictionnaire = {'Code.région': 'region',
                'unité.de.compte': 'cible',
                'POP':'pop',
                'millPOP':'millpop'}
reg=reg.rename(dictionnaire, axis = 1)
## DataFrame dep
dictionnaire = {'Code.département': 'departement',
                'Code.région': 'region',
                'unité.de.compte': 'cible',
                'POP':'pop',
                'millPOP':'millpop'}
dep=dep.rename(dictionnaire, axis = 1)
#dep.head(1)

Les codes des départements et des régions ne sont que très peu parlants. Afin de faciliter le post-processing des études, il convient de remplacer les codes par leur nom complet. 

In [ ]:
## DataFrame reg
#L'idée est ici de créer un dictionnaire clé-valeur issu d'un json (source dans le fichier README) pour "traduire" le code dans le nom de la région
json_cles_reg=pd.read_json('anciennes-nouvelles-regions.json')
dico_reg=json_cles_reg.set_index('new_code')['region'].to_dict()
reg['region']=reg['region'].map(dico_reg)
reg.tail(10)

In [ ]:
## DataFrame dep 
#On commence par modifier comme précédemment les codes des régions par leur nom 
dep['region']=dep['region'].map(dico_reg)
#Ensuite il va falloir créer un nouveau dictionnaire clé-valeur avec un fichier csv (source dans le fichier README)
csv_cles_dep=pd.read_csv('georef-france-departement.csv',sep = ';')
dico_dep=csv_cles_dep.set_index('Code Officiel Département')['Nom Officiel Département Majuscule'].to_dict()
dep['departement']=dep['departement'].map(dico_dep)
dep.head()

In [ ]:
## DataFrame commune 
csv_cles_com=pd.read_csv('v_commune_2024.csv')
dico_com=csv_cles_com.set_index('COM')['NCC'].to_dict()
commune['codgeo']=commune['codgeo'].map(dico_com)

commune.head(1)

In [ ]:
#Il convient de vérifier que les étapes précédentes n'ont pas généré de NaN 
print('Il y a',dep.isna().any(axis = 0).sum(),'colonnes contenant des valeurs manquantes dans le df departements')
print('Il y a',reg.isna().any(axis = 0).sum(),'colonnes contenant des valeurs manquantes dans le df regions')
print('Il y a',commune.isna().any(axis = 0).sum(),'colonnes contenant des valeurs manquantes dans le df communes')

In [ ]:
#On peut renommer la colonne "codgeo" qui n'est plus représentatif des variables qu'elle contient
dictionnaire = {'codgeo': 'ville',
                }
commune=commune.rename(dictionnaire, axis = 1)
commune.head(1)


# II) Statistiques exploratoires
## A) Analyse descriptive des jeux de données 
Un des objectifs sous-jacents est l'étude et le traitement des outliers
### 1) DataFrame commune

#### a) Variables quantitatives

In [ ]:
##DataFrame commune
commune.describe()
#On observe une très grand disparité dans le nombre de faits

Variable faits

In [ ]:
#On trace le diagramme en moustache qui affiche la répartition des faits
sns.boxplot(x = 'faits', data = commune)
#On observe qu'il y a 8 outliers. Afin de procéder au traitement le plus adapté, il va falloir étudier si ce sont des valeurs extrêmes ou des valeurs aberrantes. 

#Pour cela, on trie le DataFrame par nombre de faits décroissants 
comm_sorted = commune.sort_values(by = 'faits', ascending = False)
comm_sorted.head(10) 

On observe que le nombre de faits varie énormément et est très important en région Parisienne.  
Ces valeurs représentent des valeurs extrêmes et non des valeurs aberrantes. Elles sont donc gardées dans le dataset. 

Variable pop (population)

In [ ]:
#On trace le diagramme en moustache qui affiche la répartition
sns.boxplot(x = 'pop', data = commune)

#Pour cela, on trie le DataFrame par nombre de faits décroissants 
comm_sorted = commune.sort_values(by = 'pop', ascending = False)
comm_sorted.head(10) 

In [ ]:
#Etude hors Paris et Marseille
comm_sorted_out = commune.loc[~commune['ville'].isin(['PARIS', 'MARSEILLE'])].sort_values(by='pop', ascending=False)
comm_sorted_out.head(10) 

On observe des valeurs extrêmes pour la population. L'étude des populations montre que les valeurs extrêmes représentent la population Parisienne (2,1 million en 2023) et la population Marseillaise (873000 habitants en 2023) et la population Lyonnaise (522000 habiutants).
Ces valeurs ne sont donc pas des valeurs aberrantes mais constituent bien des valeurs extrêmes.

Variable tauxpourmille

In [ ]:
#On trace le diagramme en moustache qui affiche la répartition
sns.boxplot(x = 'tauxpourmille', data = commune)
#On observe qu'il y a de nombreux outliers. Afin de procéder au traitement le plus adapté, il va falloir étudier si ce sont des valeurs extrêmes ou des valeurs aberrantes. 

#Pour cela, on trie le DataFrame par nombre de faits décroissants 
comm_sorted = commune.sort_values(by = 'tauxpourmille', ascending = False)
comm_sorted.head(20) 

On remarque des valeurs extrêmes. 
Les plus grandes valeurs sont situées dans des lieux dans lesquels :
- Il y a énormément de touristes (non habitants)
- Le nombre d'habitants est faible 

Etant donné que le taux pour mille est le nombre de faits divisé par la population le tout multiplié par 1000, il est tout à fait normal que la variance du taux pour mille soit aussi grande.

#### b) Variables qualitatives

In [ ]:
#Observation du type de chaque variable
commune.dtypes
#Détermination des variables catégorielles et stockage dans un df
cat_commune=commune.select_dtypes(include='O')
#Affichage du dénombrement des différentes modalités sur les variables catégorielles à l'aide de la méthode value_counts
print(cat_commune["ville"].value_counts())
print("------------------------------")
print(cat_commune["classe"].value_counts()) #Inutile car dans tous les cas pour chaque ville chaque classe est renseignée
print("------------------------------")
print(cat_commune["cible"].value_counts())

In [ ]:
#Détermination et étude de la fréquence de chaque modalité 
print(cat_commune["ville"].value_counts(normalize = True))
print("------------------------------")
print(cat_commune["cible"].value_counts(normalize = True))

#### c) Analyse des liaisons
Pour examiner des liaisons entre des variables d'un dataset il faut distinguer trois niveaux d'analyse :
- Liaisons entre les variables quantitatives,
- Liaisons entre les variables qualitatives,
- Liaisons entre les variables qualitatives et quantitatives.

Pour chaque niveau d'analyse, on peut se poser cette question : y a-t-il dépendance ou indépendance entre les variables ?
L'objectif de cette partie est d'arriver à déterminer s'il y a une dépendance entre des variables d'un jeu de données.

Les couples de variables étudiés seront les suivants : 
- faits (variable quantitative) / pop (variable quantitative) : Test PEARSON ou SPEARMAN
- faits (variable quantitative) / annee (variable qualitative)  : Test ANOVA

Couple faits / pop : Test de corrélation de Pearson

On se demande ici si il y a une influence entre la taille de la population et le nombre de faits commis au sein d'une commune 

In [ ]:
## Mise en place du df 
com_fait_pop=commune.loc[commune['annee']== 2016].groupby(['ville','pop'])['faits'].sum().sort_values(ascending=False).reset_index()

## Hypothèses
# H0 : La taille de la population n'a pas d'influence sur le nombre de faits commis
# H1 : La taille de la population a une influence sur le nombre de faits commis : les variables sont corrélées

## Test statistique 
pearsonr(x = com_fait_pop['pop'], y = com_fait_pop['faits']) 

print("p-value: ", pearsonr(x = com_fait_pop['pop'], y = com_fait_pop['faits'])[1])
print("coefficient: ", pearsonr(x = com_fait_pop['pop'], y = com_fait_pop['faits'])[0])

#La p-value valant 0, on rejette H0 et on conclut H1. 

#sns.lmplot(x= 'pop',y = 'faits', data=com_fait_pop);
sns.lmplot(x="pop",y="faits",data=com_fait_pop, height=5, order=2, line_kws = {'color': 'red'});

D'après le test statistique de Pearson, il y a une corrélation linéaire entre la taille de la population et le nombre de faits commis au sein d'une commune.

Couple faits / année : Test de corrélation ANOVA

On se demande ici si il y a une influence entre l'évolution dans le temps (année) et le nombre de faits  

In [ ]:
## Mise en place du df 
commune_annee_fait=commune.groupby(['annee'])['faits'].sum().sort_values(ascending=False).reset_index().sort_values(by='annee')

## Hypothèses
# H0 : Il n'y a pas de relation linéaire entre l'année et le nombre de faits commis
# H1 : Il y a une relation linéaire entre l'année et le nombre de faits commis

## Test statistique 
result = statsmodels.formula.api.ols('annee ~ faits', data=commune_annee_fait).fit()
statsmodels.api.stats.anova_lm(result)

#La p-value est supérieure à 5%, on rejette H1 et on conclu H0

Tous faits confondus, il n'existe pas de relation linéaire entre l'année et le nombre de faits commis. 

Cependant, il serait intéressant d'étudier si une relation linéaire existe entre 1 type de fait précis (coups et blessures volontaires intrafamiliaux) et l'année. 

In [ ]:
## Mise en place du df 
commune_annee_fait=commune.loc[commune['classe']=='Coups et blessures volontaires intrafamiliaux'].groupby(['annee'])['faits'].sum().sort_values(ascending=False).reset_index().sort_values(by='annee')

## Hypothèses
# H0 : Il n'y a pas de relation linéaire entre l'année et le nombre de coups et blessures volontaires intrafamiliaux
# H1 : Il y a une relation linéaire entre l'année et le nombre de faits coups et blessures volontaires intrafamiliaux

## Test statistique 
result = statsmodels.formula.api.ols('annee ~ faits', data=commune_annee_fait).fit()
statsmodels.api.stats.anova_lm(result)

#La p-value est inférieure à 5%, on rejette H0 et on conclu H1

Ce test statistique montre un constat : 
Après le confinement, le nombre de coups et blessures volontaires intrafamiliaux a bondi et ne cesse de croître.

In [ ]:
sns.lineplot(x=commune_annee_fait['annee'], y=commune_annee_fait['faits'], marker = 'o', label = 'Faits');

### 2) DataFrame dep
#### a) Variables quantitatives

In [ ]:
dep.describe()

Variable faits

In [ ]:
#On trace le diagramme en moustache qui affiche la répartition des faits
sns.boxplot(x = 'faits', data = dep)
#On observe qu'il y a 8 outliers. Afin de procéder au traitement le plus adapté, il va falloir étudier si ce sont des valeurs extrêmes ou des valeurs aberrantes. 

#Pour cela, on trie le DataFrame par nombre de faits décroissants 
dep_sorted = dep.sort_values(by = 'faits', ascending = False)
dep_sorted.head(10) 

On observe que le nombre de faits varie énormément et est très important en région Parisienne. 
Ces valeurs représentent des valeurs extrêmes et non des valeurs aberrantes. Elles sont donc gardées dans le dataset.

Variable pop (population)

In [ ]:
 #On trace le diagramme en moustache qui affiche la répartition
sns.boxplot(x = 'pop', data = commune)

#Pour cela, on trie le DataFrame par nombre de faits décroissants 
dep_sorted = dep.sort_values(by = 'pop', ascending = False)
dep_sorted.head(10) 


On observe la présence d'outliers qui représentent des valeurs extrêmes et non des valeurs aberrantes. 
En effet, les outliers caractérisent le département du Nord (Ce département est le plus peublé de France avec 2,6 millions d'habitants).

Etude des densités de population

In [ ]:
dep_fait_pop=dep.groupby(['departement','pop'])['faits'].sum().sort_values(ascending=False).reset_index()

sns.kdeplot(dep_fait_pop["pop"])
plt.title('Distribution de la population dans les départements français')
plt.show();

#### b) Variables qualitatives

In [ ]:
#Observation du type de chaque variable
dep.dtypes
#Détermination des variables catégorielles et stockage dans un df
cat_dep=dep.select_dtypes(include='O')
#Affichage du dénombrement des différentes modalités sur les variables catégorielles à l'aide de la méthode value_counts
print(cat_dep["departement"].value_counts())
print("------------------------------")
print(cat_dep["classe"].value_counts()) #Inutile car dans tous les cas pour chaque ville chaque classe est renseignée
print("------------------------------")
print(cat_dep["cible"].value_counts())

In [ ]:
#Détermination et étude de la fréquence de chaque modalité
print(cat_dep["departement"].value_counts(normalize = True))
print("------------------------------")
print(cat_dep["cible"].value_counts(normalize = True))

#### c) Analyse des liaisons

Une autre manière de vérifier les relations linéaires est l'utilisation de la matrice de corrélation et le pairplot seaborn. 

In [ ]:
dep_group=dep.groupby(['annee','classe'])['faits'].sum().unstack()
dep_group.head()

In [ ]:
sns.pairplot(data=dep_group, diag_kind='kde')

### 3) DataFrame reg
#### a) Variables quantitatives

In [ ]:
reg.describe()

Variable faits

In [ ]:
#Tracé de la distribution de la variable faits
sns.boxplot(x = 'faits', data = reg)
#Sans surprise, il y a également des valeurs extrêmes en région parisienne. 
reg_sorted = reg.sort_values(by = 'faits', ascending = False)
reg_sorted.head(10) 

Variable pop

In [ ]:
 #On trace le diagramme en moustache qui affiche la répartition
sns.boxplot(x = 'pop', data = reg)

#Pour cela, on trie le DataFrame par nombre de faits décroissants 
reg_sorted = reg.sort_values(by = 'pop', ascending = False)
reg_sorted.head(10) 

Etude des densités de population

In [ ]:
reg_fait_pop=reg.groupby(['region','pop'])['faits'].sum().sort_values(ascending=False).reset_index()

sns.kdeplot(reg_fait_pop["pop"])
plt.title('Distribution de la population dans les régions français')
plt.show();

On observe une distribution avec une variance nettement moins grande que dans les DataFrames précédents. En effet, la région Ile de france est la plus peuplée avec environ 1020 habitants / km2. 

#### b) Variables qualitatives

In [ ]:
#Observation du type de chaque variable
reg.dtypes
#Détermination des variables catégorielles et stockage dans un df
cat_reg=reg.select_dtypes(include='O')
#Affichage du dénombrement des différentes modalités sur les variables catégorielles à l'aide de la méthode value_counts
print(cat_reg["region"].value_counts())
print("------------------------------")
print(cat_reg["classe"].value_counts()) #Inutile car dans tous les cas pour chaque ville chaque classe est renseignée
print("------------------------------")
print(cat_reg["cible"].value_counts())

In [ ]:
#Détermination et étude de la fréquence de chaque modalité
print(cat_reg["region"].value_counts(normalize = True))
print("------------------------------")
print(cat_reg["cible"].value_counts(normalize = True))

# III) Etudes statistiques  

Le plus intéressant dans un premier temps et d'étudier quel est le type de fait le plus commis au niveau des communes, des départements et des régions. 
Normalement, les distributions devraient être identiques (si tous les rapportages ont été correctement réalisés)

## A) Crimes et délits commis au niveau communal

In [ ]:
#Filtrage en fonction des années, regroupage par classes et somme les faits de chacune des occurences de cette classe
#Trier ensuite dans l'ordre décroissant pour un meilleur affichage de mon barplot. 
commune2016=commune.loc[commune['annee']== 2016].groupby('classe')['faits'].sum().sort_values(ascending=False)

fig, ax = plt.subplots(figsize = (6,4)) 
sns.barplot(y=commune2016.index, x=commune2016, ax=ax)
plt.title('Distribution des crimes et délits dans les communes françaises en 2016')
plt.show()

In [ ]:
commune_part=commune.loc[commune['annee']==2016].groupby(['classe','annee'])['faits'].sum().sort_values(ascending=False).reset_index()

plt.figure(figsize=(7, 7))

# Tracer le graphique en camembert sans labels
plt.pie(x=commune_part.faits, 
        labels=None,  # Pas de labels sur le graphique
        autopct=lambda x: str(round(x, 2)) + '%', 
        pctdistance=1.15, 
        wedgeprops={'linewidth': 1, 'edgecolor': 'black'})

# Ajouter la légende avec les labels
plt.legend(labels=['Vols sans violence contre des personnes',
                   'Destructions et dégradations volontaires',
                   'Coups et blessures volontaires', 'Vols dans les véhicules',
                   'Usage de stupéfiants', 'Cambriolages de logement',
                   'Coups et blessures volontaires intrafamiliaux',
                   'Vols de véhicules', 'Autres coups et blessures volontaires',
                   'Vols violents sans arme', 'Violences sexuelles',
                   "Vols d'accessoires sur véhicules", 'Trafic de stupéfiants',
                   'Vols avec armes'],
           bbox_to_anchor=(1.05, 0.5), loc='center left')
plt.title('Part des crimes et délits dans les communes françaises en 2016')
plt.show()

On observe que le délit le plus commis est le vol sans violence contre des personnes. 

On cherche maintenant à faire une étude "grosse maille" de l'évolution au fil du temps des crimes et délits, afin d'en dégager des tendances. 

In [ ]:
#Maintenant on va chercher a afficher sur le même graphique toutes les années, grâce à l'argument 'hue'
communeTest=commune.groupby(['classe','annee'])['faits'].sum().sort_values(ascending=False).reset_index().sort_values(by='annee')
#A noter que .reset_index().sort_values(by='annee') me permet de repasser sous forme d'un DataFrame. Ainsi, je peux librement utiliser l'argument 'hue' pour discrétiser en fonction des années.

sns.set_theme(style = "ticks", context = "talk", palette = "bright")
fig, ax = plt.subplots(figsize = (10,10)) 
sns.barplot(y=communeTest.classe, x=communeTest.faits, ax=ax, hue=communeTest.annee)
plt.title('Distribution et évolution au cours du temps des crimes et délits dans les communes françaises en fonction des années')
plt.show()

In [ ]:
commune_group=commune.groupby(['annee','classe'])['faits'].sum().unstack()
commune_group.plot(figsize = (20, 8), style = 'o-')
plt.title('Evolution au cours du temps des crimes et délits dans les communes françaises en fonction des années')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

On observe qu'il y a une augmentation au fil des années des coups et blessures volontaires intrafamiliaux. Cette augmentation a été beaucoup plus importante pendant et après les différents confinements. 

De même, on observe une baisse des crimes et délits lors des périodes de confinement (2019 et 2020) avant d'augmenter de nouveau. 

On observe de plus : 
- Une augmentation du trafic de stupéfiants 
- Une nette augmentation de l'usage de stupéfiants
- Une augmentation des coups et blessures volontaires
- Une très faible diminution des vols avec arme 
- Une diminution des vols violents avec arme 
- Une triste augmentation des violences sexuelles 

On se demande maintenant : 
- Quel est le top 5 des villes dans lesquelles sont commis en majorité les faits ?
- Ce classement est-il identique selon les années ? 

In [ ]:
#Année 2016
commune.head(20)
#On va sommer le nombre de faits pour chaque ville. 
commune2016Top=commune.loc[commune['annee']==2016].groupby(['ville'])['faits'].sum().sort_values(ascending = False).head(5)

sns.set_theme(style='whitegrid', palette='pastel')
fig, ax = plt.subplots(figsize = (10,4)) 
sns.barplot(x=commune2016Top.index, y=commune2016Top, ax=ax)
plt.title('Top 5 des villes dans lesquelles le plus de crimes et délits ont été commis')
plt.show()

In [ ]:
#Représentations du top 5 dans toutes les années disponibles, grâce à Plotly Express 

communeTop=commune.groupby(['annee', 'ville'])['faits'].sum().reset_index().sort_values(['annee', 'faits'], ascending=[True, False]).groupby('annee').head(5).reset_index(drop=True)

fig = px.bar(communeTop,
             x = 'ville',
             y = 'faits',
             animation_frame='annee',
             color='ville')
fig.update_layout(title_text = "Top 5 des villes dans lesquelles le plus de crimes et délits ont été commis au cours du temps",
                  width=800,
                  height=600)
fig.show('notebook')


On observe qu'entre 2016 et 2020 le classement est inchangé. Néanmoins, en 2021 et en 2023 Lille se fait devancer par Bordeaux. En 2024 le classement revient à ses valeurs initiales.
On observe de plus  en 2020 une baisse du nombre total de faits dans chacune des villes. 

Qu'en est-il de ce classement si on étudie par rapport au nombre d'habitants ?   
Pour rappel, le taux pour mille est le nombre de faits divisé par la population le tout multiplié par 1000. 

In [ ]:
#On va donc trier les villes qui ont le plus gros taux pour mille 

communeTop2=commune.groupby(['annee', 'ville'])['tauxpourmille'].sum().reset_index().sort_values(['annee', 'tauxpourmille'], ascending=[True, False]).groupby('annee').head(5).reset_index(drop=True)

#On affiche le graphique

fig = px.bar(communeTop2,
             x = 'ville',
             y = 'tauxpourmille',
             animation_frame='annee',
             color='ville')
fig.update_layout(title_text = "Top 5 des villes dans lesquelles le plus de crimes et délits ont été commis au cours du temps",
                  width=1000,
                  height=600)
fig.show('notebook')

Le graphique n'est pas aussi fluide que prévu. Néanmoins, il convient de se fier à la légende pour voir évoluer les différentes villes

Il est maintenant proposé d'étudier les différences entre la sélection du nombre de faits ou le taux pour mille habitants (donc pondéré par rapport au nombre de faits)

In [ ]:
# Création de la figure
fig = go.Figure()

# Ajout du premier tracé
fig.add_trace(go.Bar(
    x=communeTop['ville'].head(5),
    y=communeTop['faits'].head(5),
    name='Faits',
    marker_color='blue', 
))

# Ajout du second tracé
fig.add_trace(go.Bar(
    x=communeTop2['ville'].head(5),
    y=communeTop2['tauxpourmille'].head(5),
    name='Taux pour mille',
    marker_color='orange'
))

# Réglage de la taille de la figure
fig.update_layout(
    autosize=False,
    width=800,
    height=900,  
    title="Top 5 par ville",
    xaxis_title='Ville',
    yaxis_title='Valeur'
)

# Ajout des boutons pour l'interactivité
fig.update_layout(
    updatemenus=[dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=1.2,
        buttons=list([
            dict(
                label="Faits",
                method="update",
                args=[{"visible": [True, False]},
                      {"title": "Top 5 en fonction du nombre de faits (cumulés sur les années)"}]
            ),
            dict(
                label="Taux pour mille",
                method="update",
                args=[{"visible": [False, True]},
                      {"title": "Top 5 en fonction du taux pour mille (cumulés sur les années)"}]
            )
        ])
    )]
)

# Ajout des annotations
high_annotations = [dict(x=-0.05,
                         y=communeTop['faits'].mean(),
                         xanchor="right",
                         yanchor="bottom",
                         xref="x domain",
                         yref="y",
                         text="Moyenne des faits: %.2f" % communeTop['faits'].mean(),
                         showarrow=False)]

low_annotations = [dict(x=-0.05,
                        y=communeTop2['tauxpourmille'].mean(),
                        xanchor="right",
                        yanchor="bottom",
                        xref="x domain",
                        yref="y",
                        text="Moyenne des taux: %.2f" % communeTop2['tauxpourmille'].mean(),
                        showarrow=False)]

# Ajout des légendes et du titre
fig.update_layout(
    title="Comparaison des villes par nombre de faits et taux pour mille",
    xaxis_title='Villes',
    yaxis_title='Valeur'
)

# Ajout des annotations
fig.add_annotation(
    text="Sélectionnez la source des données:",
    showarrow=False,
    x=0,
    y=1.1,
    yref="paper",
    xref="paper"
)

fig.show("notebook")

Conclusion : On observe que lorsqu'on analyse sur le nombre de faits ou le taux pour mille les résultats changent. 
Le taux pour mille permet de pondérer par rapport au nombre d'habitants de la ville. Ainsi, l'étude statistique est moins sensible à la grandeur de l'échantillon.

Comme dans ma précédente hypothèse, les villes qui accueillent des touristes (exemple parfait avec Roissy et son aéroport international) remontent dans le classement. 

## B) Crimes et délits commis au niveau départemental 
 

In [ ]:
#Filtrage en fonction des années, regroupage par classes et somme les faits de chacune des occurences de cette classe
#Trier ensuite dans l'ordre décroissant pour un meilleur affichage de mon barplot. 
dep2016=dep.loc[dep['annee']== 2016].groupby('classe')['faits'].sum().sort_values(ascending=False)

fig, ax = plt.subplots(figsize = (6,4)) 
sns.barplot(y=dep2016.index, x=dep2016, ax=ax)
plt.title('Distribution des crimes et délits dans les départements français en 2016')
plt.show()

On observe que le délit le plus commis est le vols sans violence contre des personnes. 

On cherche maintenant à faire une étude "grosse maille" de l'évolution au fil du temps des crimes et délits, afin d'en dégager des tendances. 

In [ ]:
#Maintenant on va chercher a afficher sur le même graphique TOUTES les années, grâce à l'argument 'hue'
depTest=dep.groupby(['classe','annee'])['faits'].sum().sort_values(ascending=False).reset_index().sort_values(by='annee')
#A noter que .reset_index().sort_values(by='annee') me permet de repasser sous forme d'un DataFrame. Ainsi, je peux librement utiliser l'argument 'hue' pour discrétiser en fonction des années.

sns.set_theme(style = "ticks", context = "talk", palette = "bright")
fig, ax = plt.subplots(figsize = (10,10)) 
sns.barplot(y=depTest.classe, x=depTest.faits, ax=ax, hue=depTest.annee)
plt.title('Distribution et évolution au cours du temps des crimes et délits dans les départements français en fonction des années')
plt.show()

In [ ]:
dep_group=dep.groupby(['annee','classe'])['faits'].sum().unstack()
dep_group.plot(figsize = (20, 8), style = 'o-')
plt.title('Evolution au cours du temps des crimes et délits dans les départements français en fonction des années')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

Les conclusions sont identiques à l'étude des données au niveau communal. 

On observe qu'il y a une augmentation au fil des années des coups et blessures volontaires intrafamiliaux. Cette augmentation a été beaucoup plus importante pendant et après les différents confinements. 

De même, on observe une baisse des crimes et délits lors des périodes de confinement (2019 et 2020) avant d'augmenter de nouveau. 

On observe de plus : 
- Une augmentation du trafic de stupéfiants 
- Une nette augmentation de l'usage de stupéfiants
- Une augmentation des coups et blessures volontaires
- Une très faible diminution des vols avec arme 
- Une diminution des vols violents avec arme
- Une triste augmentation des violences sexuelles 

On se demande maintenant : 
- Quel est le top 5 des villes dans lesquelles sont commis en majorité les faits ?
- Ce classement est-il identique selon les années ? 

In [ ]:
#Représentations du top 5 dans toutes les années disponibles, grâce à Plotly Express 

depTop=dep.groupby(['annee', 'departement'])['faits'].sum().reset_index().sort_values(['annee', 'faits'], ascending=[True, False]).groupby('annee').head(5).reset_index(drop=True)

fig = px.bar(depTop,
             x = 'departement',
             y = 'faits',
             animation_frame='annee',
             color='departement')
fig.update_layout(title_text = "Top 5 des départements dans lesquels le plus de crimes et délits ont été commis au cours du temps",
                  width=1000,
                  height=600)
fig.show('notebook')


On observe que le classement reste identique entre 2016 et 2020. 
Les positions du Nord et des Bouches du Rhone s'inversent à partir 2021 tout comme celles du Rhone et de Seine Saint Denis. 

Qu'en est-il de ce classement si on étudie par rapport au nombre d'habitants ?   
Pour rappel, le taux pour mille est le nombre de faits divisé par la population le tout multiplié par 1000. 

In [ ]:
#On va donc trier les départements qui ont le plus gros taux pour mille 

depTop2=dep.groupby(['annee', 'departement'])['tauxpourmille'].sum().reset_index().sort_values(['annee', 'tauxpourmille'], ascending=[True, False]).groupby('annee').head(5).reset_index(drop=True)

#On affiche le graphique

fig = px.bar(depTop2,
             x = 'departement',
             y = 'tauxpourmille',
             animation_frame='annee',
             color='departement')
fig.update_layout(title_text = "Top 5 des départements dans lesquel le plus de crimes et délits ont été commis au cours du temps",
                  width=1000,
                  height=600)
fig.show('notebook')

Il est maintenant proposé d'étudier les différences entre la sélection du nombre de faits ou le taux pour mille habitants (donc pondéré par rapport au nombre de faits)

In [ ]:
# Création de la figure
fig = go.Figure()

# Ajout du premier tracé
fig.add_trace(go.Bar(
    x=depTop['departement'].head(5),
    y=depTop['faits'].head(5),
    name='Faits',
    marker_color='blue', 
))

# Ajout du second tracé
fig.add_trace(go.Bar(
    x=depTop2['departement'].head(5),
    y=depTop2['tauxpourmille'].head(5),
    name='Taux pour mille',
    marker_color='orange'
))

# Réglage de la taille de la figure
fig.update_layout(
    autosize=False,
    width=800,
    height=900,  
    title="Top 5 par département",
    xaxis_title='Département',
    yaxis_title='Valeur'
)

# Ajout des boutons pour l'interactivité
fig.update_layout(
    updatemenus=[dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=1.2,
        buttons=list([
            dict(
                label="Faits",
                method="update",
                args=[{"visible": [True, False]},
                      {"title": "Top 5 en fonction du nombre de faits (cumulés sur les années)"}]
            ),
            dict(
                label="Taux pour mille",
                method="update",
                args=[{"visible": [False, True]},
                      {"title": "Top 5 en fonction du taux pour mille (cumulés sur les années)"}]
            )
        ])
    )]
)

# Ajout des annotations
high_annotations = [dict(x=-0.05,
                         y=depTop['faits'].mean(),
                         xanchor="right",
                         yanchor="bottom",
                         xref="x domain",
                         yref="y",
                         text="Moyenne des faits: %.2f" % depTop['faits'].mean(),
                         showarrow=False)]

low_annotations = [dict(x=-0.05,
                        y=depTop2['tauxpourmille'].mean(),
                        xanchor="right",
                        yanchor="bottom",
                        xref="x domain",
                        yref="y",
                        text="Moyenne des taux: %.2f" % depTop2['tauxpourmille'].mean(),
                        showarrow=False)]

# Ajout des légendes et du titre
fig.update_layout(
    title="Comparaison des départements par nombre de faits et taux pour mille",
    xaxis_title='Région',
    yaxis_title='Valeur'
)

# Ajout des annotations
fig.add_annotation(
    text="Sélectionnez la source des données:",
    showarrow=False,
    x=0,
    y=1.1,
    yref="paper",
    xref="paper"
)

fig.show("notebook")

Conclusion : On observe que lorsqu'on analyse sur le nombre de faits ou le taux pour mille les résultats changent. 
Le taux pour mille permet de pondérer par rapport au nombre d'habitants du département. Ainsi, l'étude statistique est moins sensible à la grandeur de l'échantillon et on peut comparer département comme Paris avec un petit comme la guyane. 

On observe ici que la Seine Saint-Denis, dont les villes ne sont pas ressorties dans le top 5 de la précédente étude, ressort en 2ème position. 

## C) Crimes et délits commis au niveau régional


On observe qu'entre 2016 et 2020 le classement est inchangé. Néanmoins, en 2021 et en 2023 Lille se fait devancer par Bordeaux. En 2024 le classement revient à ses valeurs initiales.
On observe de plus  en 2020 une baisse du nombre total de faits dans chacune des villes. Cela est du au confinement.

In [ ]:
#Je filtre en fonction des années, je regroupe les classes et je somme les faits de chacune des occurences de cette classe
#Ensuite je trie dans l'ordre décroissant pour un meilleur affichage de mon barplot. 
reg2016=reg.loc[reg['annee']== 2016].groupby('classe')['faits'].sum().sort_values(ascending=False)

fig, ax = plt.subplots(figsize = (6,4)) 
sns.barplot(y=reg2016.index, x=reg2016, ax=ax)
plt.title('Distribution des crimes et délits dans les régions françaises en 2016')
plt.show()


On observe que le délit le plus commis est le vol sans violence contre des personnes ainsi que les destructions et dégradations volontaires. 

On cherche maintenant à faire une étude "grosse maille" de l'évolution au fil du temps des crimes et délits, afin d'en dégager des tendances. 

In [ ]:
#Maintenant on va chercher a afficher sur le même graphique TOUTES les années, grâce à l'argument 'hue'
regTest=reg.groupby(['classe','annee'])['faits'].sum().sort_values(ascending=False).reset_index().sort_values(by='annee')
#A noter que .reset_index().sort_values(by='annee') me permet de repasser sous forme d'un DataFrame. Ainsi, je peux librement utiliser l'argument 'hue' pour discrétiser en fonction des années.

sns.set_theme(style = "ticks", context = "talk", palette = "bright")
fig, ax = plt.subplots(figsize = (10,10)) 
sns.barplot(y=regTest.classe, x=regTest.faits, ax=ax, hue=regTest.annee)
plt.title('Distribution et évolution au cours du temps des crimes et délits dans les régions françaises en fonction des années')
plt.show()

In [ ]:
reg_group=reg.groupby(['annee','classe'])['faits'].sum().unstack()
reg_group.plot(figsize = (20, 8), style = 'o-')
plt.title('Evolution au cours du temps des crimes et délits dans les régions françaises en fonction des années')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

Les obeservations sont les mêmes que pour les deux datasets précédents. On observe qu'il y a une augmentation au fil des années des coups et blessures volontaires intrafamiliaux. Cette augmentation a été beaucoup plus importante pendant et après les différents confinements. 

De même, on observe une baisse des crimes et délits lors des périodes de confinement (2019 et 2020) avant d'augmenter de nouveau.

On observe de plus : 
- Une augmentation du trafic de stupéfiants 
- Une nette augmentation de l'usage de stupéfiants
- Une augmentation des coups et blessures volontaires
- Une très faible diminution des vols avec arme 
- Une diminution des vols violents avec arme 
- Une triste augmentation des violences sexuelles 

On se demande maintenant : 
- Quel est le top 5 des villes dans lesquelles sont commis en majorité les faits ?
- Ce classement est-il identique selon les années ? 

In [ ]:
#Représentations du top 5 dans toutes les années disponibles, grâce à Plotly Express 

regTop=reg.groupby(['annee', 'region'])['faits'].sum().reset_index().sort_values(['annee', 'faits'], ascending=[True, False]).groupby('annee').head(5).reset_index(drop=True)

fig = px.bar(regTop,
             x = 'region',
             y = 'faits',
             animation_frame='annee',
             color='region')
fig.update_layout(title_text = "Top 5 des régions dans lesquelles le plus de crimes et délits ont été commis au cours du temps",
                  width=1000,
                  height=600)
fig.show('notebook')

On observe que le classement reste identique peu importe les années. 
L'île de France est la région dans laquelle le plus de crimes et délits ont été commis. Cependant ce classement n'a rien d'étonnant par rapport à la densité de population de cette région. 

Qu'en est-il de ce classement si on étudie par rapport au nombre d'habitants ?   
Pour rappel, le taux pour mille est le nombre de faits divisé par la population le tout multiplié par 1000. 

In [ ]:
#On va donc trier les villes qui ont le plus gros taux pour mille 

regTop2=reg.groupby(['annee', 'region'])['tauxpourmille'].sum().reset_index().sort_values(['annee', 'tauxpourmille'], ascending=[True, False]).groupby('annee').head(5).reset_index(drop=True)

#On affiche le graphique

fig = px.bar(regTop2,
             x = 'region',
             y = 'tauxpourmille',
             animation_frame='annee',
             color='region')
fig.update_layout(title_text = "Top 5 des régions dans lesquelles le plus de crimes et délits ont été commis au cours du temps",
                  width=1000,
                  height=600)
fig.show('notebook')

Il est maintenant proposé d'étudier les différences entre la sélection du nombre de faits ou le taux pour mille habitants (donc pondéré par rapport au nombre de faits).

In [ ]:
# Création de la figure
fig = go.Figure()

# Ajout du premier tracé
fig.add_trace(go.Bar(
    x=regTop['region'].head(5),
    y=regTop['faits'].head(5),
    name='Faits',
    marker_color='blue', 
))

# Ajout du second tracé
fig.add_trace(go.Bar(
    x=regTop2['region'].head(5),
    y=regTop2['tauxpourmille'].head(5),
    name='Taux pour mille',
    marker_color='orange'
))

# Réglage de la taille de la figure
fig.update_layout(
    autosize=False,
    width=800,
    height=900,  
    title="Top 5 par région",
    xaxis_title='Région',
    yaxis_title='Valeur'
)

# Ajout des boutons pour l'interactivité
fig.update_layout(
    updatemenus=[dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=1.2,
        buttons=list([
            dict(
                label="Faits",
                method="update",
                args=[{"visible": [True, False]},
                      {"title": "Top 5 en fonction du nombre de faits (cumulés sur les années)"}]
            ),
            dict(
                label="Taux pour mille",
                method="update",
                args=[{"visible": [False, True]},
                      {"title": "Top 5 en fonction du taux pour mille (cumulés sur les années)"}]
            )
        ])
    )]
)

# Ajout des annotations
high_annotations = [dict(x=-0.05,
                         y=regTop['faits'].mean(),
                         xanchor="right",
                         yanchor="bottom",
                         xref="x domain",
                         yref="y",
                         text="Moyenne des faits: %.2f" % regTop['faits'].mean(),
                         showarrow=False)]

low_annotations = [dict(x=-0.05,
                        y=regTop2['tauxpourmille'].mean(),
                        xanchor="right",
                        yanchor="bottom",
                        xref="x domain",
                        yref="y",
                        text="Moyenne des taux: %.2f" % regTop2['tauxpourmille'].mean(),
                        showarrow=False)]

# Ajout des légendes et du titre
fig.update_layout(
    title="Comparaison des régions par nombre de faits et taux pour mille",
    xaxis_title='Région',
    yaxis_title='Valeur'
)

# Ajout des annotations
fig.add_annotation(
    text="Sélectionnez la source des données:",
    showarrow=False,
    x=0,
    y=1.1,
    yref="paper",
    xref="paper"
)

fig.show("notebook")

Conclusion : On observe que lorsqu'on analyse sur le nombre de faits ou le taux pour mille les résultats changent. 
Le taux pour mille permet de pondérer par rapport au nombre d'habitants de la région. Ainsi, l'étude statistique est moins sensible à la grandeur de l'échantillon et on peut comparer une grande région comme l'ile de France avec une petite. 

On observe ici aussi que ce sont les régions dans lesquelles il y a le plus de mouvement de population (tourisme par exemple) qui sont en tête. 
Il serait intéressant de corréler cette hypothèse avec une étude statistique complémentaire sur le tourisme en France. 

# IV) Représentations géographiques

L'objectif de cette partie est d'utiliser le package GeoPandas pour afficher une carte. 

GeoPandas nécessite un GeoDataFrame dans lequel une colonne "geometry" contient les informations pour créer la carte
Cette étude portera sur les données du DataFrame commune.  


## A) Vue départementale

In [ ]:
#Import d'un geojson incluant les géométries 
json_cles_geo=gpd.read_file('departements.geojson')

json_cles_geo['nom']=json_cles_geo['nom'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
json_cles_geo['nom']=json_cles_geo['nom'].apply(lambda nom : nom.upper()) #On met les noms en majuscule
json_cles_geo['nom'] = json_cles_geo['nom'].str.replace("-", " ") #On enleve les tirets pour la concordance avec les valeurs du df dep
json_cles_geo['nom'] = json_cles_geo['nom'].str.replace("'", " ") #On enleve les apostrophes pour la concordance avec les valeurs du df dep
json_cles_geo.head()


In [ ]:
#Pour cette étude, on se concentre exclusivement sur la France métropolitaine. 
#On supprime donc toutes les lignes des DOM TOMS dont les regions sont : 'GUADELOUPE', 'MARTINIQUE', 'GUYANE', 'LA REUNION', 'MAYOTTE'
dep = dep[~dep['region'].isin(['GUADELOUPE', 'MARTINIQUE', 'GUYANE', 'LA REUNION', 'MAYOTTE'])]
dep.region.unique()

In [ ]:
#On créée 1 nouvelle colonne dans dep pouvant accueillir les valeurs de géométrie
dep['geometry']= 0
dep.head(1)

In [ ]:
#On cherche maintenant à ajouter ces colonnes au DataFrame dep. 
#On utilise un dictionnaire clé-valeur pour associer les bonnes coordonnées en fonction du nom du département

dico_geo=json_cles_geo.set_index('nom')['geometry'].to_dict()
dep['geometry']=dep['departement'].map(dico_geo)

dep.head()

In [ ]:
#On vérifie qu'on a pas créé de nans 
dep.isna().any(axis = 0) 
display(pd.DataFrame(dep.isna().sum(), columns=["Nombre de NA"]))  #Aucun NANs, on peut continuer l'analyse la tête haute



In [ ]:
#On transforme le DataFrame en GeoDataFrame
dep=gpd.GeoDataFrame(dep, geometry = 'geometry')
#On fixe un premier CRS qu'on fera évoluer pour en étudier l'influence
dep=dep.set_crs('wgs84', allow_override=True)
print(dep.crs)

In [ ]:
#On affiche le total du nombre de faits, toute année cumulées 
depTotal=dep.groupby(['departement','geometry'])['faits'].sum().sort_values(ascending=False).reset_index()
depTotal=gpd.GeoDataFrame(depTotal, geometry = 'geometry')
#depTotal.plot(column= 'faits', cmap='viridis', legend=True, figsize=(10,10), linewidth=1)
depTotal.explore(column= 'faits', cmap='viridis', legend=True)